In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

%load_ext autoreload
%autoreload

In [2]:
from collections import defaultdict

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
import lightgbm as lgb
from tqdm import tqdm

from src.data import get_sequences, load_data, download_movielens1m, ValSASRecDataset, TrainSASRecDataset
from src.utils import fix_seed
from src.model import SASRec

# CONFIG

In [3]:
seed = 0
max_len = 200
batch_size = 128
embedding_size = 64

n_ext_users = 10

In [4]:
  
fix_seed(seed)

# Подгрузка данных

In [ ]:
# Скачиваем и загружаем данные
download_movielens1m("data/movielens_1m")
ratings = load_data("data/movielens_1m")

ratings = ratings[ratings["rating"] > 3.5]

# Подготовка данных
num_users = ratings['user_id'].nunique()
num_items = ratings['item_id'].nunique()

user2id = {val:i for i, val in enumerate(ratings['user_id'].unique())}
item2id = {val:i+1 for i, val in enumerate(ratings['item_id'].unique())}

ratings['user_id'] = ratings['user_id'].map(user2id)
ratings['item_id'] = ratings['item_id'].map(item2id)

Dataset already exists.


In [29]:
bs, seq_len, n_ext_users, emb_size = 2, 3, 4, 5

obj = torch.rand([bs, seq_len, n_ext_users, emb_size])
int_obj = torch.rand([bs, seq_len, emb_size])

dot_prod = torch.matmul(obj, int_obj.unsqueeze(-2).transpose(-2, -1))
print(dot_prod.shape)
softmax_dot_prod = nn.functional.softmax(dot_prod, -2)
print(softmax_dot_prod.shape, obj.shape)
agg_embeddings = (softmax_dot_prod * obj).sum(dim=2)
agg_embeddings.shape

torch.Size([2, 3, 4, 1])
torch.Size([2, 3, 4, 1]) torch.Size([2, 3, 4, 5])


torch.Size([2, 3, 5])

In [ ]:
int_obj_one = int_obj[0, 0]
ext_obj_one = obj[0, 0]
# int_obj_one.tile(obj.shape[2], 2).shape, int_obj.shape

print(int_obj_one.tile((len(ext_obj_one), 1)).shape, int_obj_one.shape)

int_obj.unsqueeze(2).tile((1, 1, obj.shape[2], 1)).shape, int_obj.shape,

concated = torch.cat((ext_obj_one, int_obj_one.tile((len(ext_obj_one), 1))), axis=1)
concated.shape

torch.Size([4, 5]) torch.Size([5])


torch.Size([4, 10])

In [6]:
# # Создаем последовательности
# sequences = []
# times = []
# for user_id, group in ratings.groupby('user_id'):
#     group = group.sort_values('timestamp')
#     user_seq = group['item_id'].tolist()
#     user_times = group['timestamp'].astype(int).tolist()
#     sequences.append(user_seq)
#     times.append(user_times)

# # Разделяем на train/val
# split_idx = int(0.8 * len(sequences))
# train_sequences = sequences[:split_idx]
# val_sequences = sequences[split_idx:]

# train_times = times[:split_idx]
# val_times = times[split_idx:]

# all_users = ratings['user_id'].unique()
# train_users, val_users = all_users[:split_idx], all_users[split_idx:]
# print(len(train_users), len(val_users))

(train_sequences, val_sequences), (train_times, val_times), (train_users, val_users) = get_sequences(ratings)

# Обучение нейронки

In [7]:
# Создание датасета для SASRec

# class TrainSASRecDataset(Dataset):
#     def __init__(self, sequences, times, max_len=50):
#         self.sequences = sequences
#         self.times = times
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         seq = self.sequences[idx]
#         time = self.times[idx]
#         if len(seq) > self.max_len:
#             seq = seq[-self.max_len:]
#             time = time[-self.max_len:]
#         else:
#             seq = [0] * (self.max_len - len(seq)) + seq
#             time = [0] * (self.max_len - len(time)) + time
#         input_seq = torch.tensor(seq[:-1], dtype=torch.long)
#         input_times = torch.tensor(time[:-1], dtype=torch.long)
#         target = torch.tensor(seq[1:], dtype=torch.long)
#         return (input_seq, input_times), target


# class ValSASRecDataset(Dataset):
#     def __init__(self, sequences, times, max_len=50):
#         self.sequences = sequences
#         self.times = times
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         seq = self.sequences[idx]
#         time = self.times[idx]
#         if len(seq) > self.max_len:
#             seq = seq[-self.max_len:]
#             time = time[-self.max_len:]
#         else:
#             seq = [0] * (self.max_len - len(seq)) + seq
#             time = [0] * (self.max_len - len(time)) + time
#         input_seq = torch.tensor(seq[:-1], dtype=torch.long)
#         input_times = torch.tensor(time[:-1], dtype=torch.long)
#         target = torch.tensor(seq[-1], dtype=torch.long)

#         return (input_seq, input_times), target



train_dataset = TrainSASRecDataset(train_sequences, train_times, max_len)
val_dataset = ValSASRecDataset(val_sequences, val_times, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
# # Модель SASRec на PyTorch
# class SASRec(nn.Module):
#     def __init__(self,
#         num_items,
#         hidden_units=64,
#         num_heads=2,
#         num_blocks=2,
#         dropout_rate=0.2,
#         max_len=200,
#         ext_flag=False
#     ):
#         super(SASRec, self).__init__()
#         self.num_items = num_items
#         self.hidden_units = hidden_units
#         self.max_len = max_len
#         self.ext_flag = ext_flag

#         self.item_emb = nn.Embedding(num_items + 1, hidden_units, padding_idx=0)
#         self.pos_emb = nn.Embedding(max_len, hidden_units)
#         self.dropout = nn.Dropout(dropout_rate)

#         self.encoder_layers = nn.ModuleList([
#             nn.TransformerEncoderLayer(
#                 d_model=hidden_units,
#                 nhead=num_heads,
#                 dim_feedforward=hidden_units,
#                 dropout=dropout_rate,
#                 batch_first=True
#             ) for _ in range(num_blocks)
#         ])

#         self.layer_norm = nn.LayerNorm(hidden_units)
#         self.output_layer = nn.Linear(hidden_units, num_items + 1)

#     def forward(self, input_seqs, timestamps=None):
#         batch_size, seq_len = input_seqs.size()

#         # Position encoding
#         positions = torch.arange(seq_len, dtype=torch.long, device=input_seqs.device)
#         positions = positions.unsqueeze(0).expand(batch_size, seq_len)

#         # Item and position embedding
#         item_emb = self.item_emb(input_seqs)
#         pos_emb = self.pos_emb(positions)
#         x = item_emb + pos_emb
#         x = self.dropout(x)

#         # Transformer encoder
#         mask = self.generate_square_subsequent_mask(seq_len).to(input_seqs.device)
#         for layer in self.encoder_layers:
#             x = layer(x, mask)

#         x = self.layer_norm(x)

#         if self.ext_flag:
#             ext_context = self.get_external_features(timestamps).to(x.device)
#             extended_data = torch.cat([x, ext_context], dim=2)
#             output = self.ext_head(extended_data)
#         else:
#             output = self.output_layer(x)

#         return output

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def get_embeddings(self, input_seqs):
#         batch_size, seq_len = input_seqs.size()

#         positions = torch.arange(seq_len, dtype=torch.long, device=input_seqs.device)
#         positions = positions.unsqueeze(0).expand(batch_size, seq_len)

#         item_emb = self.item_emb(input_seqs)
#         pos_emb = self.pos_emb(positions)
#         x = item_emb + pos_emb

#         mask = self.generate_square_subsequent_mask(seq_len).to(input_seqs.device)
#         for layer in self.encoder_layers:
#             x = layer(x, mask)

#         x = self.layer_norm(x)
#         return x

#     def freeze(self):
#         for param in self.parameters():
#             param.requires_grad = False
#         print("Все слои сети заморожены.")

#     def add_external_features(self, ext_features):
#         self.ext_flag = True
#         self.freeze()
#         self.ext_head = nn.Linear(self.hidden_units*2, num_items + 1)
#         self.time_list = ext_features[0]
#         self.ext_embeddings = ext_features[1]

#     def get_external_features(self, timestamps):
#         bs, seq_len = timestamps.shape
#         timestamps = timestamps.reshape(-1).cpu().detach().numpy()

#         ext_ids = np.searchsorted(self.time_list, timestamps, side='right') - 1
#         ext_context = self.ext_embeddings[ext_ids]
#         ext_context = torch.tensor(ext_context, dtype=torch.float32).reshape(bs, seq_len, -1)
#         return ext_context

#     # ----------------------------------------
#     # def add_external_features(self, ext_features):
#     #     self.ext_flag = True
#     #     self.freeze()
#     #     self.ext_head = nn.Linear(self.hidden_units*2, num_items + 1)
#     #     self.ext_features = ext_features
#     #     self.ext_features['timestamp'] = self.ext_features['timestamp'].astype(int)

#     # def get_external_features(self, timestamps):
#     #     bs, seq_len = timestamps.shape
#     #     timestamps = timestamps.reshape(-1)
#     #     ext_context = []
#     #     for time in timestamps:
#     #         ext_context.append(self.get_one_ext_represent(time))

#     #     ext_context = torch.tensor(ext_context, dtype=torch.float32)
#     #     ext_context = ext_context.reshape(bs, seq_len, -1)
#     #     return ext_context

#     # def get_one_ext_represent(self, time):
#     #     context_info = self.ext_features[self.ext_features['timestamp'] <= time.cpu().detach().numpy()]
#     #     if len(context_info) > 0:
#     #         context_info = context_info.iloc[-1]
#     #         mean_context = context_info['mean_embedding']
#     #         return mean_context
#     #     else:
#     #         return np.zeros(self.hidden_units)


In [9]:
def calculate_ndcg(model, dataloader, device, k=10):
    model.eval()

    # Вместо накопления всех предсказаний, будем вычислять NDCG по батчам
    ndcg_scores = []

    with torch.no_grad():
        for (input_seqs, input_times), targets in dataloader:
            input_seqs, input_times, targets = input_seqs.to(device), input_times.to(device), targets.to(device)
            outputs = model(input_seqs, input_times)
            last_outputs = outputs[:, -1, :]

            # Преобразуем в numpy
            predictions = last_outputs.cpu().numpy()
            targets_np = targets.cpu().numpy()

            # Создаем матрицу релевантности только для текущего батча
            n_items = predictions.shape[1]
            relevance = np.zeros((len(targets_np), n_items))
            relevance[np.arange(len(targets_np)), targets_np] = 1

            # Вычисляем NDCG для текущего батча
            try:
                batch_ndcg = ndcg_score(relevance, predictions, k=k)
                ndcg_scores.append(batch_ndcg)
            except MemoryError:
                # Если батч слишком большой, разбиваем его на подбатчи
                sub_batch_size = 100
                for i in range(0, len(targets_np), sub_batch_size):
                    end_idx = min(i + sub_batch_size, len(targets_np))
                    sub_relevance = relevance[i:end_idx]
                    sub_predictions = predictions[i:end_idx]
                    sub_ndcg = ndcg_score(sub_relevance, sub_predictions, k=k)
                    ndcg_scores.append(sub_ndcg)

    # Возвращаем среднее значение NDCG по всем батчам
    return np.mean(ndcg_scores)

# Обучение модели SASRec
def train_sasrec(model, train_loader, val_loader, optimizer, criterion, device, epochs=10):
    model.to(device)
    best_ndcg = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch_idx, ((input_seqs, input_times), targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            input_seqs, input_times, targets = input_seqs.to(device), input_times.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(input_seqs, input_times)

            # Получаем предсказания для последнего элемента в последовательности
            # last_outputs = outputs[:, -1, :]
            #loss = criterion(last_outputs, targets)
            outputs, targets = outputs.reshape(-1, num_items+1), targets.reshape(-1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Валидация
        val_ndcg = calculate_ndcg(model, val_loader, device)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Val NDCG@{10}: {val_ndcg:.4f}")

        # Сохраняем лучшую модель
        if val_ndcg > best_ndcg:
            best_ndcg = val_ndcg
            torch.save(model.state_dict(), "best_sasrec_model.pth")

    return model

In [10]:
# Инициализируем модель
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SASRec(num_items, embedding_size=embedding_size, max_len=max_len).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# # # Обучаем модель
# model = train_sasrec(model, train_loader, val_loader, optimizer, criterion, device, epochs=1)

In [11]:
# # Загружаем лучшую модель
# model.load_state_dict(torch.load("best_sasrec_model.pth"))

# Аггрегации

In [12]:
# Загружаем лучшую модель
model.load_state_dict(torch.load("best_sasrec_model.pth"))

<All keys matched successfully>

In [13]:
user_subset = np.random.choice(train_users, n_ext_users, replace=False)
subset_data = ratings[ratings['user_id'].isin(user_subset)].sort_values(['user_id', 'timestamp'])

# Создаем словарь для хранения эмбеддингов пользователей по времени
user_embeddings = defaultdict(list)
#time_embeddings = defaultdict(list)
all_timestamps = set()

# Получаем эмбеддинги для каждого пользователя в каждый момент времени
model.eval()
with torch.no_grad():
    for user_id in tqdm(user_subset, desc="Processing users"):
        user_data = subset_data[subset_data['user_id'] == user_id]
        user_items = user_data['item_id'].tolist()
        timestamps = user_data['timestamp'].astype(int).tolist()

        if len(user_items) <= model.max_len:
            seq = [0] * (model.max_len - len(user_items)) + user_items
            input_seq = torch.tensor([seq], dtype=torch.long).to(device)
            embeddings = model.get_internal_embeddings(input_seq).cpu().numpy()[0, :, :]

        else:
            for i in range(0, len(user_items) + 1, model.max_len):
                seq = user_items[i:model.max_len + i]
                input_seq = torch.tensor([seq], dtype=torch.long).to(device)
                batch_emb = model.get_internal_embeddings(input_seq).cpu().numpy()[0, :, :]
                if i > 0:
                    embeddings = np.concatenate([embeddings, batch_emb], axis=0)
                else:
                    embeddings = batch_emb

        for time, embed in zip(timestamps, embeddings):
            all_timestamps.add(time)
            user_embeddings[user_id].append((time, embed))
            #time_embeddings[time].append((user_id, embed))

all_timestamps = sorted(list(all_timestamps))

Processing users: 100%|██████████| 10/10 [00:00<00:00, 37.92it/s]


In [14]:
# Создаем таблицу с агрегированными эмбеддингами по времени
# all_timestamps = sorted(set([ts for user_embs in user_embeddings.values() for ts, _ in user_embs]))

agg_raw_data_embeddings = []

for timestamp in tqdm(all_timestamps, desc="Aggregating embeddings"):
    # Собираем последние эмбеддинги для каждого пользователя на данный момент времени
    current_embeddings = []
    current_times = []
    for user_id, embeddings_list in user_embeddings.items():
        # Находим последний эмбеддинг пользователя до данного времени
        user_embs_before = [(ts, emb) for ts, emb in embeddings_list if ts <= timestamp]
        if user_embs_before:
            # Берем последний эмбеддинг
            time, last_emb = user_embs_before[-1]
            current_embeddings.append(np.array(last_emb))
            current_times.append(time)

    # Агрегируем эмбеддинги всех пользователей
    if len(current_embeddings) < n_ext_users:
        n_embeddings = len(current_embeddings)
        for i in range(n_ext_users - n_embeddings):
            current_embeddings.append(np.zeros(embedding_size))
            current_times.append(-1e7)
    agg_raw_data_embeddings.append([timestamp, current_times, current_embeddings])


agg_raw_data_embeddings = sorted(agg_raw_data_embeddings, key = lambda d: d[0])

time_list = np.array([data[0] for data in agg_raw_data_embeddings])
time_to_embeddings = np.array([data[1] for data in agg_raw_data_embeddings])
ext_embeddings = np.array([data[2]  for data in agg_raw_data_embeddings])

Aggregating embeddings: 100%|██████████| 356/356 [00:00<00:00, 23446.22it/s]


In [15]:
time_list.shape, time_to_embeddings.shape, ext_embeddings.shape

((356,), (356, 10), (356, 10, 64))

In [16]:
# # Загружаем лучшую модель
# from src.model import SASRec
# model = SASRec(num_items, embedding_size=embedding_size, max_len=max_len).to(device)
# model.load_state_dict(torch.load("best_sasrec_model.pth"))

In [17]:
external_features = time_list, ext_embeddings

model.add_external_features(time_list, time_to_embeddings, ext_embeddings)

Все слои сети заморожены.


In [18]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model = train_sasrec(model, train_loader, val_loader, optimizer, criterion, device, epochs=2)

Epoch 1/2: 100%|██████████| 38/38 [00:29<00:00,  1.30it/s]


Epoch 1, Loss: 5.5699, Val NDCG@10: 0.0047


Epoch 2/2: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]


Epoch 2, Loss: 3.4089, Val NDCG@10: 0.0071


In [19]:
128*199*64

1630208

In [20]:
# if self.pooling_type == "mean":
#     pooled_vector = torch.Tensor(np.mean(vectors, axis=0))

# elif self.pooling_type == "max":
#     pooled_vector = torch.Tensor(np.max(vectors, axis=0))

# elif "attention" in self.pooling_type:
#     vectors = torch.Tensor(vectors).to(user_emb.device)

#     if "learnable_attention" in self.pooling_type:
#         if not self.learnable_attention_matrix:
#             raise ValueError("Learnable attention matrix wasn't initialized!")
#         vectors_prep = self.learnable_attention_matrix(vectors)
#     elif self.pooling_type == "symmetrical_attention":
#         if not self.learnable_attention_matrix:
#             raise ValueError("Learnable attention matrix wasn't initialized!")
#         vectors_prep = self.learnable_attention_matrix(vectors)
#         user_emb = self.learnable_attention_matrix(user_emb)
#     elif self.pooling_type == "kernel_attention":
#         if not self.attention_kernel:
#             raise ValueError("Attention kernel wasn't initialized!")
#         vectors_prep = self.attention_kernel(vectors)
#         user_emb = self.attention_kernel(user_emb)
#     else: vectors_prep = vectors
#     dot_prod = vectors_prep @ user_emb.unsqueeze(0).transpose(1, 0)
#     softmax_dot_prod = nn.functional.softmax(dot_prod, 0)
#     if "attention_hawkes" in self.pooling_type:
#         times = torch.Tensor((times - time) * self.exp_param).to(user_emb.device)
#         times = times.unsqueeze(-1)
#         time_part = torch.exp(times)
#         softmax_dot_prod = softmax_dot_prod * time_part
#     pooled_vector = (softmax_dot_prod * vectors).sum(dim=0)

# elif self.pooling_type == "exp_hawkes":
#     pooled_vector = np.mean(vectors * np.exp((times - time) * self.exp_param).reshape(-1,1), axis=0)
#     pooled_vector = torch.Tensor(pooled_vector).to(user_emb.device)
    
# elif "hawkes" in self.pooling_type:
#     if not self.hawkes_nn:
#         raise ValueError("Hawkes NN wasn't initialized!")
    
#     vectors = torch.Tensor(vectors).to(user_emb.device)
#     concated = torch.cat((vectors, user_emb.tile((len(vectors), 1))), axis=1)
#     emb_part = self.hawkes_nn(concated)
#     times = torch.Tensor((times - time) * self.exp_param).to(user_emb.device)
#     times = times.unsqueeze(-1)
#     if self.pooling_type == "learnable_hawkes":
#         if not self.hawkes_time_nn:
#             raise ValueError("Hawkes NN for time wasn't initialized!")
#         time_part = self.hawkes_time_nn(times)
#     elif self.pooling_type == "exp_learnable_hawkes":
#         time_part = torch.exp(times)
#     else:
#         raise ValueError("Unsupported pooling type.")
#     pooled_vector = torch.sum(emb_part * time_part, axis=0) 

# else:
#     raise ValueError("Unsupported pooling type.")

# device = next(self.parameters()).device

In [21]:
# Агрегируем эмбеддинги по времени
def aggregate_embeddings(embeddings_list, method='mean'):
    if not embeddings_list:
        return np.zeros(model.hidden_units)

    embeddings = np.array([emb[1] for emb in embeddings_list])
    if method == 'mean':
        return np.mean(embeddings, axis=0)
    elif method == 'max':
        return np.max(embeddings, axis=0)
    else:
        raise ValueError(f"Unknown aggregation method: {method}")

# Создаем таблицу с агрегированными эмбеддингами по времени
all_timestamps = sorted(set([ts for user_embs in user_embeddings.values() for ts, _ in user_embs]))

aggregated_embeddings = []

for timestamp in tqdm(all_timestamps, desc="Aggregating embeddings"):
    # Собираем последние эмбеддинги для каждого пользователя на данный момент времени
    current_embeddings = []
    for user_id, embeddings_list in user_embeddings.items():
        # Находим последний эмбеддинг пользователя до данного времени
        user_embs_before = [(ts, emb) for ts, emb in embeddings_list if ts <= timestamp]
        if user_embs_before:
            # Берем последний эмбеддинг
            last_emb = user_embs_before[-1][1]
            current_embeddings.append(last_emb)

    # Агрегируем эмбеддинги всех пользователей
    if current_embeddings:
        mean_agg = aggregate_embeddings([(timestamp, emb) for emb in current_embeddings], 'mean')
        max_agg = aggregate_embeddings([(timestamp, emb) for emb in current_embeddings], 'max')

        aggregated_embeddings.append({
            'timestamp': timestamp,
            'mean_embedding': mean_agg,
            'max_embedding': max_agg
        })

# Создаем DataFrame с агрегированными эмбеддингами
agg_emb_df = pd.DataFrame(aggregated_embeddings)

Aggregating embeddings: 100%|██████████| 356/356 [00:00<00:00, 13365.67it/s]


In [22]:
agg_emb_df.head()

,timestamp,mean_embedding,max_embedding
0,963085003000000000,"[1.2166228, -0.7584174, -0.72425693, 0.8039133...","[1.2166228, -0.7584174, -0.72425693, 0.8039133..."
1,963085239000000000,"[1.3693753, -0.7254764, -0.7280423, 0.7639481,...","[1.3693753, -0.7254764, -0.7280423, 0.7639481,..."
2,963085379000000000,"[1.2751011, -0.72293264, -0.739836, 0.72587556...","[1.2751011, -0.72293264, -0.739836, 0.72587556..."
3,963085444000000000,"[1.304151, -0.73856103, -0.7569565, 0.7945317,...","[1.304151, -0.73856103, -0.7569565, 0.7945317,..."
4,963085516000000000,"[1.2577156, -0.7414157, -0.74677974, 0.7025273...","[1.2577156, -0.7414157, -0.74677974, 0.7025273..."


In [23]:
agg_emb_df = agg_emb_df.sort_values('timestamp')
time_list = agg_emb_df['timestamp'].values
ext_embeddings = agg_emb_df['mean_embedding'].values
ext_embeddings = np.stack(ext_embeddings)

external_features = time_list, ext_embeddings

In [24]:
model.add_external_features(external_features)

TypeError: add_external_features() missing 2 required positional arguments: 'time_to_embeddings' and 'ext_features'

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model = train_sasrec(model, train_loader, val_loader, optimizer, criterion, device, epochs=20)

In [ ]:
def check_unfrozen_params(model):
    unfrozen_params = []
    for name, param in model.named_parameters():
        if param.requires_grad:
            unfrozen_params.append(name)
    return unfrozen_params

print("Размороженные параметры:", check_unfrozen_params(model))

Размороженные параметры: ['ext_head.weight', 'ext_head.bias']


# Boosting

In [ ]:
# Добавляем расчет NDCG для бустинга
def calculate_ndcg_for_boosting(model, X, y, user_ids, k=10):
    """
    Вычисляет NDCG@k для модели бустинга, группируя предсказания по пользователям
    """
    # Получаем предсказания модели
    predictions = model.predict(X)

    # Создаем DataFrame с user_id, предсказаниями и истинными значениями
    results_df = pd.DataFrame({
        'user_id': user_ids,
        'prediction': predictions,
        'true_rating': y
    })

    # Группируем по пользователям
    ndcg_scores = []
    for user_id, group in results_df.groupby('user_id'):
        if len(group) < 2:
            continue  # Пропускаем пользователей с менее чем 2 взаимодействиями

        # Сортируем предсказания и истинные значения
        pred_sorted = group.sort_values('prediction', ascending=False)['true_rating'].values
        true_sorted = group.sort_values('true_rating', ascending=False)['true_rating'].values

        # Вычисляем NDCG
        ndcg = ndcg_score([true_sorted], [pred_sorted], k=k)
        ndcg_scores.append(ndcg)

    return np.mean(ndcg_scores) if ndcg_scores else 0

In [ ]:
boost_data = []
boost_labels = []
boost_user_ids = []  # Для хранения ID пользователей
boost_timestamps = []  # Для хранения временных меток

add_external = False

with torch.no_grad():
    for user_id in tqdm(ratings["user_id"].unique(), desc="Preparing boosting data"):
        user_data = subset_data[subset_data['user_id'] == user_id]
        user_items = user_data['item_id'].tolist()
        user_ratings = user_data['rating'].tolist()
        user_timestamps = user_data['timestamp'].tolist()

        for i in range(1, len(user_items)):
            # Получаем эмбеддинг пользователя
            seq = user_items[:i]
            if len(seq) > model.max_len:
                seq = seq[-model.max_len:]
            else:
                seq = [0] * (model.max_len - len(seq)) + seq

            input_seq = torch.tensor([seq], dtype=torch.long).to(device)
            user_embedding = model.get_embeddings(input_seq)[0, -1, :].cpu().numpy()

            # Получаем эмбеддинг айтема
            item_id = user_items[i]
            item_embedding = model.item_emb(torch.tensor([item_id], dtype=torch.long).to(device))[0].cpu().numpy()

            # Получаем агрегированный эмбеддинг контекста
            timestamp = user_data.iloc[i-1]['timestamp']
            meta_features = np.array([user_id, item_id, timestamp])
            n_meta_features = len(meta_features)

            if add_external:
                context_info = agg_emb_df[agg_emb_df['timestamp'] <= timestamp].iloc[-1]
                mean_context = context_info['mean_embedding']
                max_context = context_info['max_embedding']

                # Объединяем все эмбеддинги
                combined_features = np.concatenate([meta_features, user_embedding, mean_context, max_context, item_embedding])
            else:
                combined_features = np.concatenate([meta_features, user_embedding, item_embedding]) #mean_context, max_context, ])

            boost_data.append(combined_features)
            boost_labels.append(user_ratings[i])
            boost_user_ids.append(user_id)
            boost_timestamps.append(user_timestamps[i])

# Преобразуем в numpy массивы
X = np.array(boost_data)
y = np.array(boost_labels)
user_ids = np.array(boost_user_ids)
train_users_boost = np.array([data[0] in train_users for data in X])

# Разделяем на train/test
# split_idx = int(0.8 * len(X))
X_train, X_test = X[train_users_boost, n_meta_features:], X[~train_users_boost, n_meta_features:]
y_train, y_test = y[train_users_boost], y[~train_users_boost]
user_ids_train, user_ids_test = user_ids[train_users_boost], user_ids[~train_users_boost]

# Обучаем модель бустинга
print("Training LightGBM model...")
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05)
lgb_model.fit(X_train, y_train)

# Оцениваем модель с помощью RMSE
train_pred = lgb_model.predict(X_train)
test_pred = lgb_model.predict(X_test)

train_rmse = np.sqrt(np.mean((train_pred - y_train) ** 2))
test_rmse = np.sqrt(np.mean((test_pred - y_test) ** 2))

print(f"LightGBM Train RMSE: {train_rmse:.4f}")
print(f"LightGBM Test RMSE: {test_rmse:.4f}")

# Оцениваем модель с помощью NDCG
train_ndcg = calculate_ndcg_for_boosting(lgb_model, X_train, y_train, user_ids_train, k=10)
test_ndcg = calculate_ndcg_for_boosting(lgb_model, X_test, y_test, user_ids_test, k=10)

print(f"LightGBM Train NDCG@10: {train_ndcg:.4f}")
print(f"LightGBM Test NDCG@10: {test_ndcg:.4f}")

Preparing boosting data: 100%|██████████| 6038/6038 [01:37<00:00, 61.99it/s]  


Training LightGBM model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060908 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32625
[LightGBM] [Info] Number of data points in the train set: 43823, number of used features: 128
[LightGBM] [Info] Start training from score 4.400338
